In [1]:
import pandas as pd

In [2]:
runs_df = pd.read_csv('wandb/wandb_runs/wandb_runs_results.csv')
runs_df.head(10)

,Name,Notes,model_name,per_device_train_batch_size,gradient_accumulation_steps,ID,State,User,Group,Job Type,...,local_rope_theta,mlp_bias,mlp_dropout,norm_bias,norm_eps,repad_logits_with_grad,sparse_pred_ignore_index,sparse_prediction,eval/evaluation_loss,eval/margin_ranking_loss
0,astral-cloud-143,-,SOBertLarge,1.0,4.0,anx6f5hs,crashed,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,likely-resonance-142,-,SOBertLarge,1.0,4.0,hs9zlvj3,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,daily-bush-139,-,SOBertLarge,1.0,4.0,pe1glus9,crashed,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.12156
3,graceful-fog-137,-,SOBertBase,1.0,4.0,fy2r99gc,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.45470
4,tough-wildflower-136,-,ModernBERT-base,1.0,4.0,7ahxwweu,finished,NaN,NaN,NaN,...,10000.0,False,0.0,False,0.00001,False,-100.0,False,NaN,NaN
5,decent-frost-131,-,NaN,NaN,NaN,vsii1ux1,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,splendid-waterfall-130,-,NaN,NaN,NaN,yfddkmiz,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,genial-yogurt-129,-,SOBertLarge,NaN,NaN,dovez39l,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,easy-thunder-128,-,SOBertLarge,NaN,NaN,kqu7t7ug,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,warm-hill-127,-,SOBertLarge,NaN,NaN,lub8jqha,finished,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
metadata_cols = ['Name', 'Notes', 'model_name', 'Tags']
metric_cols = [*[f'ndcg@{k}_g.exponential_d.logarithmic (Max)' for k in range(1, 11)], 'hit_rate@1 (Max)', 'mae (Min)']
metric_cols += [f'eval/{metric_col}' for metric_col in metric_cols]
runs_df = runs_df[[*metadata_cols, *metric_cols]]

for k in range(1, 11):
    ndcg_col = f'ndcg@{k}_g.exponential_d.logarithmic (Max)'
    runs_df.loc[runs_df[ndcg_col].isna(), ndcg_col] = runs_df.loc[runs_df[ndcg_col].isna(),
    f'eval/ndcg@{k}_g.exponential_d.logarithmic (Max)']
runs_df.loc[runs_df['hit_rate@1 (Max)'].isna(), 'hit_rate@1 (Max)'] = runs_df.loc[runs_df['hit_rate@1 (Max)'].isna(),
'eval/hit_rate@1 (Max)']
runs_df.loc[runs_df['mae (Min)'].isna(), 'mae (Min)'] = runs_df.loc[runs_df['mae (Min)'].isna(), 'eval/mae (Min)']

cols_to_drop = [col for col in runs_df.columns if col.startswith('eval/')]
runs_df = runs_df.drop(columns=cols_to_drop)
runs_df = runs_df.rename(columns={f'ndcg@{k}_g.exponential_d.logarithmic (Max)': f'ndcg@{k} (Max)' for k in range(1, 11)})
metric_cols = [col for col in runs_df.columns if col not in metadata_cols]
for metric_col in metric_cols:
    runs_df[metric_col] = runs_df[metric_col].apply(lambda x: f'{x:.4f}'[:-1])

runs_df.head()

,Name,Notes,model_name,Tags,ndcg@1 (Max),ndcg@2 (Max),ndcg@3 (Max),ndcg@4 (Max),ndcg@5 (Max),ndcg@6 (Max),ndcg@7 (Max),ndcg@8 (Max),ndcg@9 (Max),ndcg@10 (Max),hit_rate@1 (Max),mae (Min)
0,astral-cloud-143,-,SOBertLarge,cross-encoder,0.537,0.634,0.710,0.762,0.783,0.793,0.797,0.799,0.801,0.801,0.423,2.209
1,likely-resonance-142,-,SOBertLarge,cross-encoder,0.524,0.624,0.703,0.756,0.778,0.788,0.791,0.794,0.795,0.795,0.407,2.246
2,daily-bush-139,-,SOBertLarge,"cross-encoder, ranking",0.540,0.639,0.710,0.762,0.786,0.795,0.799,0.801,0.802,0.803,0.427,na
3,graceful-fog-137,-,SOBertBase,"bi-encoder, ranking",0.534,0.638,0.712,0.764,0.784,0.795,0.798,0.800,0.801,0.802,0.416,na
4,tough-wildflower-136,-,ModernBERT-base,cross-encoder,0.516,0.614,0.694,0.751,0.772,0.782,0.787,0.790,0.791,0.792,0.401,2.220


In [4]:
runs_df.to_csv('outputs/tables/test_wandb_metrics.csv', index=False)